<a href="https://colab.research.google.com/github/amitbhola20/Plant-Disease-Identification-Models/blob/main/VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import os
import seaborn as sns
from keras.preprocessing import image
from keras import applications
import tensorflow as tf
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
import itertools
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Read input images and assign labels based on folder names
print(os.listdir("/content/drive/My Drive/Research/AgricultureDataset/"))

In [ ]:
# Resize images
SIZE = 224

In [ ]:
#Capture training data and labels into respective lists
train_images = []
train_labels = []

for directory_path in glob.glob("/content/drive/MyDrive/Research/Data/train/*"):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*")):
        # print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

#Convert lists to arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
# Capture test/validation data and labels into respective lists

test_images = []
test_labels = []
for directory_path in glob.glob("/content/drive/MyDrive/Research/Data/test/*"):
    crop_label = directory_path.split("/")[-1]
    print(crop_label)
    for img_path in glob.glob(os.path.join(directory_path, "*")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(crop_label)

#Convert lists to arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
X_train, y_train, X_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
 #Normalize pixel values to between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

In [ ]:
#Load model without classifier/fully connected layers
model_VGG19 = tf.keras.applications.VGG19(input_shape=(224, 224,3), weights='imagenet', include_top=False)

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in model_VGG19.layers:
	layer.trainable = False

model_VGG19.summary()  #Trainable parameters will be 0

#Now, let us use features from convolutional network
feature_extractor=model_VGG19.predict(x_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_training = features #This is our X input to ML models.
X_for_training.shape

#Send test data through same feature extractor process
X_test_feature = model_VGG19.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

# Machine Learning Models

###Random forest

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
RF_model.fit(X_for_training, y_train)

In [ ]:
#Now predict using the trained RF model.
RF_prediction = RF_model.predict(X_test_features)
#Inverse le transform to get original label back.
RF_prediction = le.inverse_transform(RF_prediction)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, RF_prediction))

In [ ]:
#Confusion Matrix - verify accuracy of each class
print('Classification Report')
cm = confusion_matrix(test_labels, RF_prediction)
print(cm)
sns.heatmap(cm, annot=True)

  KNN

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [ ]:
# Train the model on training data
knn_model.fit(X_for_training, y_train)

In [ ]:
#Now predict using the trained knn model.
knn_prediction = knn_model.predict(X_test_features)
#Inverse le transform to get original label back.
knn_prediction = le.inverse_transform(knn_prediction)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, knn_prediction))

In [ ]:
#Confusion Matrix - verify accuracy of each class
print('Classification Report')
cm = confusion_matrix(test_labels, knn_prediction)
print(cm)
sns.heatmap(cm, annot=True)

###XGBoost

In [ ]:
#XGBOOST
import xgboost as xgb
XB = xgb.XGBClassifier()

In [ ]:
# Train the model on training data
XB_model.fit(X_for_training, y_train)

In [ ]:
#Now predict using the trained XB model.
XB_prediction = XB_model.predict(X_test_features)
#Inverse le transform to get original label back.
XB_prediction = le.inverse_transform(XB_prediction)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, XB_prediction))

In [ ]:
#Confusion Matrix - verify accuracy of each class
print('Classification Report')
cm = confusion_matrix(test_labels, XB_prediction)
print(cm)
sns.heatmap(cm, annot=True)

###Naive Bayes

In [ ]:
#NAIVE BAYES
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()

In [ ]:
# Train the model on training data
NB_model.fit(X_for_training, y_train)

In [ ]:
#Now predict using the trained NB model.
NB_prediction = NB_model.predict(X_test_features)
#Inverse le transform to get original label back.
NB_prediction = le.inverse_transform(NB_prediction)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, NB_prediction))

In [ ]:
#Confusion Matrix - verify accuracy of each class
print('Classification Report')
cm = confusion_matrix(test_labels, NB_prediction)
print(cm)
sns.heatmap(cm, annot=True)

###SVM

SVM linear

In [ ]:
from sklearn.svm import SVC
SVM_linear = SVC(kernel = 'linear').fit(X_train, y_train)

In [ ]:
# Train the model on training data
SVM_linear_model.fit(X_for_training, y_train)

In [ ]:
#Now predict using the trained SVM_linear model.
SVM_linear_prediction = SVM_linear_model.predict(X_test_features)
#Inverse le transform to get original label back.
SVM_linear_prediction = le.inverse_transform(SVM_linear_prediction)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, SVM_linear_prediction))

In [ ]:
#Confusion Matrix - verify accuracy of each class
print('Classification Report')
cm = confusion_matrix(test_labels, SVM_linear_prediction)
print(cm)
sns.heatmap(cm, annot=True)

SVM poly

In [ ]:
from sklearn.svm import SVC
SVM_poly = SVC(kernel = 'poly').fit(X_train, y_train)

In [ ]:
# Train the model on training data
SVM_poly_model.fit(X_for_training, y_train)

In [ ]:
#Now predict using the trained SVM_poly model.
SVM_poly_prediction = SVM_poly_model.predict(X_test_features)
#Inverse le transform to get original label back.
SVM_poly_prediction = le.inverse_transform(SVM_poly_prediction)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, SVM_poly_prediction))

In [ ]:
#Confusion Matrix - verify accuracy of each class
print('Classification Report')
cm = confusion_matrix(test_labels, SVM_poly_prediction)
print(cm)
sns.heatmap(cm, annot=True)

SVM rbf

In [ ]:
from sklearn.svm import SVC
SVM_rbf = SVC(kernel = 'rbf').fit(X_train, y_train)

In [ ]:
# Train the model on training data
SVM_rbf_model.fit(X_for_training, y_train)

In [ ]:
#Now predict using the trained SVM_rbf model.
SVM_rbf_prediction = SVM_rbf_model.predict(X_test_features)
#Inverse le transform to get original label back.
SVM_rbf_prediction = le.inverse_transform(SVM_rbf_prediction)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, SVM_rbf_prediction))

In [ ]:
#Confusion Matrix - verify accuracy of each class
print('Classification Report')
cm = confusion_matrix(test_labels, SVM_rbf_prediction)
print(cm)
sns.heatmap(cm, annot=True)

###Decision Tree

In [ ]:
#DECISION TREE
from sklearn.ensemble import DecisionTreeClassifier
DT_model = DecisionTreeClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
DT_model.fit(X_for_training, y_train)

In [ ]:
#Now predict using the trained DT model.
DT_prediction = DT_model.predict(X_test_features)
#Inverse le transform to get original label back.
DT_prediction = le.inverse_transform(DT_prediction)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, DT_prediction))

In [ ]:
#Confusion Matrix - verify accuracy of each class
print('Classification Report')
cm = confusion_matrix(test_labels, DT_prediction)
print(cm)
sns.heatmap(cm, annot=True)

In [ ]:
#Print Train Accuracy
dt_train_accuracy = DecisionTree.score(X_train,y_train)
print("Training accuracy = ",DecisionTree.score(X_train,y_train))
#Print Test Accuracy
dt_test_accuracy = DecisionTree.score(X_test,y_test)
print("Testing accuracy = ",DecisionTree.score(X_test,y_test))